In [15]:
class Website:

    def __init__(self, name, url, targetPattern, absoluteUrl, presidentTag, titleTag, dateTag, bodyTag):
        self.name = name
        self.url = url
        self.targetPattern = targetPattern
        self.absoluteUrl = absoluteUrl
        self.presidentTag = presidentTag
        self.titleTag = titleTag
        self.dateTag = dateTag
        self.bodyTag = bodyTag
        


class Content:

    def __init__(self, url, president, title, date, body):
        self.url = url
        self.president = president
        self.title = title
        self.date = date
        self.body = body

    #To store contents scraped into MYSQL
    def store(self):
        cur.execute('INSERT INTO pages (url, president, title, date, body) VALUES ''("%s", "%s", "%s", "%s", "%s")', (self.url, self.president, self.title, self.date, self.body))
        cur.connection.commit()
        

    def print(self):
        print(f'URL: {self.url}')
        print(f'PRESIDENT: {self.president}')
        print(f'TITLE: {self.title}')
        print(f'DATE: {self.date}')
        print(f'BODY:\n{self.body}')
        
        

In [16]:
import re
from bs4 import BeautifulSoup
from urllib.request import urlopen
from urllib.error import HTTPError
from urllib.error import URLError
import requests
import pymysql


#creates MYSQL connection
#input your mysql password in passwd
print('before')
conn = pymysql.connect(host='127.0.0.1',
user='root', passwd='', db='mysql', charset='utf8')
cur = conn.cursor()
cur.execute("USE scraping")

class Crawler:
    def __init__(self, site):
        self.site = site
        self.visited = {}

    def getPage(url, headers):
        session = requests.Session()
        try:
            html = session.get(url, headers=headers)
        except Exception as e:
            return e
            
        return BeautifulSoup(html.text, 'html.parser')

    def safeGet(bs, selector):
        selectedElems = bs.select(selector)
        if selectedElems is not None and len(selectedElems) > 0:
            return '\n'.join([elem.get_text() for elem in selectedElems])
        return ''

    def getContent(self, url):
        """
        Extract content from a given page URL
        """
        headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36"}
        bs = Crawler.getPage(url, headers=headers)
        if bs is not None:
            president = Crawler.safeGet(bs, self.site.presidentTag)
            title = Crawler.safeGet(bs, self.site.titleTag)
            date = Crawler.safeGet(bs, self.site.dateTag)
            body = Crawler.safeGet(bs, self.site.bodyTag)
           
            return Content(url, president, title, date, body)
        return Content(url, '', '')

    def crawl(self):
        """
        Get pages from website home page
        """
        headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36"}
        bs = Crawler.getPage(self.site.url, headers=headers)
        #print(bs)
        targetPages = bs.table.find_all('a', href=re.compile(self.site.targetPattern))
        #print(targetPages)
        for targetPage in targetPages:
            #print(targetPages)
            url = targetPage.attrs['href']
            print(url)
            url = url if self.site.absoluteUrl else f'{self.site.url}{targetPage}'
            #print(url)
            if url not in self.visited:
                self.visited[url] = self.getContent(url)
                self.visited[url].store()


brookings = Website('The APP', 'https://presidency.ucsb.edu/documents/presidential-documents-archive-guidebook/annual-messages-congress-the-state-the-union#Table of SOTU', '(/documents/)[A-Za-z0-9-]+[state-the-union]+[A-Za-z0-9-]+', True, 'h3.diet-title', 'h1', 'div.field-docs-start-date-time', 'div.field-docs-content')
crawler = Crawler(brookings)
try:
   crawler.crawl()
finally:
    cur.close()
    conn.close()

before
https://www.presidency.ucsb.edu/documents/address-before-joint-session-the-congress-3
https://www.presidency.ucsb.edu/documents/address-before-joint-session-the-congress-the-state-the-union-28
https://www.presidency.ucsb.edu/documents/address-before-joint-session-the-congress-the-state-the-union-29
https://www.presidency.ucsb.edu/documents/address-before-joint-session-the-congress-the-state-the-union-30
https://www.presidency.ucsb.edu/documents/address-before-joint-session-the-congress-2
https://www.presidency.ucsb.edu/documents/address-before-joint-session-the-congress-the-state-the-union-25
https://www.presidency.ucsb.edu/documents/address-before-joint-session-the-congress-the-state-the-union-26
https://www.presidency.ucsb.edu/documents/address-before-joint-session-the-congress-the-state-the-union-27
https://www.presidency.ucsb.edu/documents/address-before-joint-session-congress-the-state-the-union-2
https://www.presidency.ucsb.edu/documents/address-before-joint-session-the-co